In [9]:
import os
import re
import json
from pydub import AudioSegment
import librosa
from openai import OpenAI
import numpy as np
import pandas as pd

In [5]:
client = OpenAI(api_key=os.getenv('OPEN_AI_KEY'))

## Importa o audio da live

In [6]:
path_audio = "Y:\Transcription lives\live_Enthusia_Professional_Racing_voice_isolation.mp3"

## Faz a edição dos trechos em silencio

In [10]:
# Extrair o audio a partir do video
audio_clip = AudioSegment.from_file(path_audio, "mp3")

# Capturar a taxa de amostragem (sample rate) do áudio
sample_rate = audio_clip.frame_rate

# Caminho para salvar o arquivo de áudio extraído temporariamente
temp_audio_path = './temp_audio.wav'

# Exportar o áudio para um arquivo temporário
audio_clip.export(temp_audio_path, format="wav")

# Carregar o arquivo de áudio temporário com librosa
y, sr = librosa.load(temp_audio_path, sr=sample_rate)  # y é o sinal de áudio, sr é a taxa de amostragem

# Calcular a energia do sinal de áudio (pode ser usado como uma proxy para volume)
energy = librosa.feature.rms(y=y)

# Definir um limiar de energia para considerar como silêncio
# quanto menor o valor, mais sensível será a detecção de silêncio. Logo terá trechos mais longos de diálogos.
threshold = 0.005  

# Encontrar os frames que estão acima do limiar de energia
non_silent_frames = np.nonzero(energy > threshold)[1]

# Converter os frames para tempos de início e fim dos trechos não silenciosos
non_silent_times = librosa.frames_to_time(non_silent_frames, sr=sr)

# definir tolerancia do que é silencio (em segundos) pra não cortar falas.
tolerancia_silencio = 2

# Criar uma tabela para armazenar os trechos não silenciosos
tabela_trechos_naosilenciosos = pd.DataFrame()


# Agrupar os tempos em segmentos contínuos de áudio não silencioso
non_silent_segments = []
start_time = non_silent_times[0]
for i in range(1, len(non_silent_times)):
    if non_silent_times[i] > non_silent_times[i - 1] + tolerancia_silencio:
        end_time = non_silent_times[i - 1]
        non_silent_segments.append((start_time, end_time))
        start_time = non_silent_times[i]

# Adicionar o último segmento
non_silent_segments.append((start_time, non_silent_times[-1]))

# Imprimir os segmentos não silenciosos
for start_time, end_time in non_silent_segments:
    pd_linha = pd.DataFrame(data={'Inicio': [start_time],
                                  'Fim': [end_time]})

    tabela_trechos_naosilenciosos = pd.concat([tabela_trechos_naosilenciosos, pd_linha],
                                              axis=0,
                                              ignore_index=True)

print("Trechos com dialogos:")
print(tabela_trechos_naosilenciosos)
os.remove(temp_audio_path)

# Salvando os trechos não silenciosos em um arquivo de áudio
# Inicializar um áudio segmento vazio para concatenar os trechos
combined_audio = AudioSegment.silent(duration=0)

# Iterar sobre os trechos não silenciosos e concatená-los
for index, row in tabela_trechos_naosilenciosos.iterrows():
    # Converter os tempos de início e fim de segundos para milissegundos
    start_time_ms = int(row['Inicio'] * 1000)
    end_time_ms = int(row['Fim'] * 1000)

    # Extrair o trecho de áudio correspondente
    audio_chunk = audio_clip[start_time_ms:end_time_ms]

    # Concatenar o trecho ao áudio combinado
    combined_audio += audio_chunk

# Caminho para salvar o arquivo de áudio combinado
combined_audio_path = './output/live.mp3'

# Exportar o áudio combinado para um arquivo
try:
    combined_audio.export(combined_audio_path, format="mp3")
except FileNotFoundError:
    os.makedirs(os.path.dirname(combined_audio_path))
    combined_audio.export(combined_audio_path, format="mp3")


Trechos com dialogos:
          Inicio          Fim
0       0.597333   147.829333
1     150.165333   164.298667
2     167.616000   167.626667
3     174.794667   176.458667
4     185.749333   188.213333
..           ...          ...
391  4426.112000  4431.797333
392  4435.018667  4436.608000
393  4440.032000  4462.442667
394  4467.968000  4483.040000
395  4486.709333  4490.080000

[396 rows x 2 columns]


In [12]:
audio_file = open(combined_audio_path, "rb")
#audio_file = open(path_audio, "rb")

## divida o arquivo em 10 partes iguais

In [16]:
partes = []

# Carregar o arquivo de áudio
audio = AudioSegment.from_file(path_audio)

# Duração total do áudio em milissegundos
duration = len(audio)

# Duração de cada parte do áudio
part_duration = duration // 10

try:
    os.makedirs("Y:\Transcription lives\data")
except:
    print("Pasta já criada")

# Dividir o áudio em 10 partes e salvar
for part in range(10):
    start_time = part * part_duration
    end_time = (part + 1) * part_duration if (part < 9) else duration
    new_clip = audio[start_time:end_time]
    partes.append(new_clip)
    if part < 9:
        new_clip.export(f"Y:\Transcription lives\data\part_0{part+1}.mp3", format="mp3")
    else:
        new_clip.export(f"Y:\Transcription lives\data\part_{part+1}.mp3", format="mp3")
    print(f"Parte {part+1} do audio exportada com sucesso")

Parte 1 do audio exportada com sucesso
Parte 2 do audio exportada com sucesso
Parte 3 do audio exportada com sucesso
Parte 4 do audio exportada com sucesso
Parte 5 do audio exportada com sucesso
Parte 6 do audio exportada com sucesso
Parte 7 do audio exportada com sucesso
Parte 8 do audio exportada com sucesso
Parte 9 do audio exportada com sucesso
Parte 10 do audio exportada com sucesso


In [18]:
# Diretório onde os arquivos MP3 estão localizados
directory = 'Y:/Transcription lives/data'

# Lista para armazenar os caminhos dos arquivos MP3
mp3_files = []

# Listar todos os arquivos no diretório
for file in os.listdir(directory):
    if file.endswith(".mp3"):
        mp3_files.append(os.path.join(directory, file))

## Exportando a transcrição de cada parte
Uma live de 2h vai custar aproximadamente $0,60 USD ~ $0,80 pra transcrever.

In [19]:
transcricoes = {}

for mp3_file in mp3_files:
    audio_file = open(mp3_file, "rb")
    file_name = mp3_file.split('\\')[-1]
    transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file, response_format="verbose_json")
    audio_transcription = transcript.text
    transcricoes[file_name] = audio_transcription + "/n/n"
    print(f"Audio {file_name} transcrito com sucesso")

Audio part_01.mp3 transcrito com sucesso
Audio part_010.mp3 transcrito com sucesso
Audio part_02.mp3 transcrito com sucesso
Audio part_03.mp3 transcrito com sucesso
Audio part_04.mp3 transcrito com sucesso
Audio part_05.mp3 transcrito com sucesso
Audio part_06.mp3 transcrito com sucesso
Audio part_07.mp3 transcrito com sucesso
Audio part_08.mp3 transcrito com sucesso
Audio part_09.mp3 transcrito com sucesso


In [20]:
transcricoes

{'part_01.mp3': 'Olá pra você que tá ligado aqui no Flaggamer. Nesse feriadão do dia 2 de novembro nós estamos ao vivo pra vocês. Hoje vamos ter mais uma live retrô, inclusive é um jogo que o público do canal diversas vezes durante as lives já pediu para que eu trouxesse aqui por causa da simulação que esse jogo consegue proporcionar por causa da sua física realista do carro e é um jogo de Playstation 2. Inclusive essa semana eu tive todo o trabalho de configurar o volante no emulador do Playstation 2, o PCSX2. Consegui configurar o volante e então com conta disso hoje nós vamos jogar Entusia Professional Racing, um jogo da Konami lançado em 2005 que tinha como objetivo ser um dos concorrentes do Gran Turismo 4 que também tinha lançado no Japão um ano antes e agora lançou nesse momento da live. Então vai ser simplesmente espetacular essa live de hoje, espero que vocês gostem muito. Já estou com as minhas luvas da BAUGER já posicionadas, luvas profissionais, se você quiser utilizar, com

## Exportar o dicionário para um arquivo JSON

In [21]:
with open("Y:/Transcription lives/data/transcription.json", 'w', encoding='utf-8') as json_file:
    json.dump(transcricoes, json_file, ensure_ascii=False, indent=4)

## Agora vai resumir cada parte e exportar em texto
Resumir as 10 partes da live custou aproximadamente USD 0,30

In [30]:
system_msg = "Sua missão é resumir e listar os principais pontos que o Flagamer mencionou em cada parte da live em forma de um texto impessoal e predominantemente descritivo."

In [31]:
output_texto = open("Y:\Transcription lives\data\output.txt", "w")

for chave, valor in transcricoes.items():
    # print(chave, valor)
    user_msg = f"{valor}"
    output_texto.write(f"\nResumo {chave}\n:")
    response = client.chat.completions.create(model="gpt-4o",
                                       messages=[{"role": "system", "content":system_msg},
                                                {"role": "user", "content":user_msg}],
                                       temperature=0.25)
    output_texto.write(response.choices[0].message.content)
    output_texto.write("\n")
    print(f"Resumo da parte {chave} gerado com sucesso")
    
output_texto.close()
print("Resumo da live gerado com sucesso")

part_01.mp3 Olá pra você que tá ligado aqui no Flaggamer. Nesse feriadão do dia 2 de novembro nós estamos ao vivo pra vocês. Hoje vamos ter mais uma live retrô, inclusive é um jogo que o público do canal diversas vezes durante as lives já pediu para que eu trouxesse aqui por causa da simulação que esse jogo consegue proporcionar por causa da sua física realista do carro e é um jogo de Playstation 2. Inclusive essa semana eu tive todo o trabalho de configurar o volante no emulador do Playstation 2, o PCSX2. Consegui configurar o volante e então com conta disso hoje nós vamos jogar Entusia Professional Racing, um jogo da Konami lançado em 2005 que tinha como objetivo ser um dos concorrentes do Gran Turismo 4 que também tinha lançado no Japão um ano antes e agora lançou nesse momento da live. Então vai ser simplesmente espetacular essa live de hoje, espero que vocês gostem muito. Já estou com as minhas luvas da BAUGER já posicionadas, luvas profissionais, se você quiser utilizar, comprar,

Resumo da parte part_02.mp3 gerado com sucesso
part_03.mp3 Que bicho inusitado, apartando aí por uma ponte. Opa! Eita, perdi o ponto. O azul vai embora. Espero que não seja um fogate, porque senão aí é fim de corrida para mim. Ih, sai de frente! Sai de frente! Sai de frente! Volta! Joga a chicane. Muito boa a dirigibilidade desse jogo. Você tem que travar o G29 em 360 graus para você conseguir pilotar esse game. Por enquanto tá bastante bacana. Bastante bacana aqui o jogo. Vamos ver. Primeira vitória do FlyGamer. O total de colisões. Já me colidi um pouquinho mais nessa prova aí. Sai por volta. Pessoal falando que é um Bugatti EB. Tem um replay, ó. Se quiser ver o replay da corrida. Qualzinho no Gran Turismo. Quando eu falei ali na largada que o Bugatti azul foi embora, foi nesse exato momento. Você pode salvar o replay ou não. Vai definindo o sorteio. Você pode ter o seu próximo carro. João Lucas chegou aqui na live. Vamos ver. PT Cruiser. Mais um carrinho. João Lucas falando. Salve, 

Resumo da parte part_06.mp3 gerado com sucesso
part_07.mp3 Vamos lá, vamos para mais uma corrida. Chegamos aqui no canal. Eu já mostrei boa parte das pistas, aqui é de novatos, já vai começar a repetir pistas. A gente tem 3.7% de possibilidade de ganhar, vamos ver em mim mesmo. Sim Ricardo, estou emulando o jogo. Essa é a outra câmera do jogo. Eu não sabia da existência desse jogo. Jogo da Konami, criado em 2005, concorrente do Gran Turismo 4, só que acho que vendeu pouco. O foco dele não é na parte da pista do game, então tem um monte de carros e pistas e o jogo é um pouco mais complicado do que o Honda Civic. Entendeu, eu tentei me concentrar aqui, mas na época mais fechadinha, segundo o colocado tinha o mesmo carro que eu, mas ali eu superar o Ford GT já seria um pouco otimismo demais da minha parte. Vamos ver se eu ganho esse carro, vamos lá. Ganhei o Ford GT! Que cagão é esse Flag Gamer? Caramba, ganhei o Ford GT, que legal, que legal. Estou tudo demais. Agora a apelação vai começ

## Agora vai pegar essas 10 partes e fazer um texto corrido

In [32]:
output_texto = open("Y:\Transcription lives\data\output.txt", "r")
texto_corrido = output_texto.readlines()
texto = ''.join(texto_corrido)

In [33]:
texto

'\nResumo part_01.mp3\n:Flagamer inicia a live no feriado de 2 de novembro, destacando que será uma live retrô com o jogo Entusia Professional Racing, um título de Playstation 2 lançado pela Konami em 2005. Ele menciona que configurou um volante no emulador PCSX2 para proporcionar uma experiência mais realista, algo que o público do canal havia pedido devido à física realista do jogo. \n\n- **Configuração e Equipamento**: Flagamer configurou um volante no emulador PCSX2 e está usando luvas profissionais da BAUGER, promovendo um link de compra com desconto para os espectadores.\n\n- **Interação com o Público**: Agradece aos membros do canal e incentiva novos espectadores a se tornarem membros, destacando benefícios como acesso antecipado a vídeos, grupos exclusivos no WhatsApp e Telegram, e participação prioritária em corridas dos inscritos.\n\n- **Apresentação do Jogo**: Inicia o jogo Entusia Professional Racing, destacando a música eletrônica da intro e comentando sobre a física reali

In [34]:
system_msg = "Sua missão é fazer um texto corrido de até 4000 caracteres sobre os principais pontos das 10 partes da live com estilo de texto descritivo. A parte 10 da live deve ser obrigatoriamente o parágrafo final do texto"

In [35]:
user_msg = f"{texto}"
response = client.chat.completions.create(model="gpt-4o-mini",
                                   messages=[{"role": "system", "content":system_msg},
                                            {"role": "user", "content":user_msg}],
                                   temperature=0.6)

In [36]:
print(response.choices[0].message.content)

A live do Flagamer, realizada em um feriado, trouxe um olhar nostálgico sobre o jogo "Entusia Professional Racing", um título de corrida do PlayStation 2 que, apesar de seu lançamento em 2005, ainda cativa os entusiastas do gênero. O streamer começou a transmissão configurando um volante no emulador PCSX2, uma escolha que prometia uma experiência mais imersiva, especialmente considerando a física realista do jogo. Com luvas profissionais da BAUGER, ele não apenas buscou proporcionar uma jogabilidade autêntica, mas também interagiu com o público, promovendo um link de compra para os espectadores.

O jogo foi apresentado com entusiasmo, destacando a música eletrônica da introdução e a variedade de modos disponíveis, como o modo carreira "Intuition Life". Flagamer fez questão de explicar as mecânicas do jogo, como os pontos entusiastas que se perdem com erros de pilotagem, enfatizando a importância de dirigir com precisão para evitar desclassificações. Durante a primeira corrida, ele perc

In [37]:
output_texto.close()
with open("Y:/Transcription lives/data/output.txt", "a") as texto:
    texto.write('\n'+ response.choices[0].message.content + '\n')